In [ ]:
import numpy as np
import pandas as pd
from iholt import Holt_model
import os
from tqdm import trange
import json

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [ ]:
n_clusters = 2
# method = 'kmeans'
method = 'hierarchical/euclidean'
data_set = 'Irish_2010'
month = 2

In [ ]:
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'interval', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)

attr = pd.read_csv(os.path.join(path, 'data', f'{data_set}_attr_final.csv'))

In [ ]:
series = []
for i in trange(len(attr)):
    id = attr['ID'][i]
    df = pd.read_csv(os.path.join(path, 'data', f'{data_set}_monthly_interval', f'{id}.csv'), header = None).values
    df = df[(month-1)*2:month*2, :months[month-1]*24]
    series.append(df)
series = np.array(series)

In [ ]:
for i in range(n_clusters):
    index = list(clusters[month-1] == i)
    sub_series = series[index]
    sub_series = np.sum(sub_series, axis=0)
    test = sub_series[:, -168:]
    train = sub_series[:, :-168]
    
    scale = np.zeros(2)
    scale[0] = np.max(train)
    scale[1] = np.min(train)
    train = (train - scale[1])/(scale[0] - scale[1])
    test = (test - scale[1])/(scale[0] - scale[1])   
                        
    # Model
    # build model
    holt_model = Holt_model(train)

    # optimize
    bnds = [[0, 1]] * 8
    x0 = np.ones(8) * 0.5   # Parameters [a11, a12, a21, a22, b11, b12, b21, b22]
    result = holt_model.train(x0, bnds)

    # predict
    It, Lt, Tt = holt_model.pred(result.x, 168, test)

    pred_series = np.squeeze(np.array(It)).T[:, -168:]
    print('cluster:', i, 'train status:', result.success)
    
    break